In [2]:
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns

In [210]:
import os

print(os.getcwd())

c:\Users\Администратор\Documents\study\CSME 830\house_price_project\notebooks


In [3]:
data = pd.read_csv(r'c:\Users\Администратор\Documents\study\CSME 830\house_price_project\data\raw\sample_data.csv') #reading the sample dataframe of cleaned data from main.ipynb 
data.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price,has_prev_sale
0,sold,1,2,0.21,Coral Gables,Florida,33134.0,847.000000,2022-02-15,330000.0,1
1,sold,4,3,3.00,Colleyville,Texas,76034.0,2758.000000,2022-03-09,1275000.0,1
2,for_sale,2,1,0.21,New York,New York,10003.0,965.063941,2014-05-15,800000.0,1
3,for_sale,3,3,0.10,Elgin,South Carolina,29045.0,1726.000000,NaN,306830.0,0
4,sold,4,3,0.42,Wilmington,Delaware,19807.0,4327.000000,2022-01-05,600000.0,1


In [212]:
# data.columns.tolist()
df = data
data.shape

(140951, 11)

## Data Inspection.

In [213]:
df.describe()

,bed,bath,acre_lot,zip_code,house_size,price,has_prev_sale
count,140951.000000,140951.000000,140951.000000,140951.000000,140951.000000,1.409510e+05,140951.000000
mean,3.267306,2.495924,10.421729,52159.122511,2039.080874,5.859180e+05,0.766437
std,1.222335,1.207854,698.334033,29707.580974,1348.297662,1.313216e+06,0.423099
min,1.000000,1.000000,0.000000,602.000000,100.000000,0.000000e+00,0.000000
25%,3.000000,2.000000,0.150000,28639.000000,1305.000000,2.349000e+05,1.000000
50%,3.000000,2.000000,0.210000,48706.000000,1757.000000,3.750000e+05,1.000000
75%,4.000000,3.000000,0.380000,78572.500000,2431.000000,5.999000e+05,1.000000
max,20.000000,20.000000,100000.000000,99999.000000,50000.000000,1.350000e+08,1.000000


In [214]:
df['zip_code'] = df['zip_code'].astype(int)

In [215]:
# convert the column's type to date time
df['prev_sold_date'] = pd.to_datetime(df['prev_sold_date'])

In [216]:
# create a new feature to calculate the year since the last sale
df['years_since_prev_sale'] = (
    pd.Timestamp('today') - df['prev_sold_date']
).dt.days / 365

#those records with no sales will be filled with 0
df['years_since_prev_sale'] = df['years_since_prev_sale'].fillna(0)

#drop the old feature 'prev_sold_date' 
df = df.drop('prev_sold_date', axis=1)

In [217]:
# # Plot Size vs Price to see if outliers make sense
# sns.scatterplot(data=df, x='acre_lot', y='price')

In [218]:
def n_outliers(df, col_x):
    # 1. Calculate the corners of the "box"
    Q1 = df[col_x].quantile(0.25)
    Q3 = df[col_x].quantile(0.75)
    IQR = Q3 - Q1

    # 2. Define what counts as an outlier (standard rule is 1.5 * IQR)
    lower_limit = Q1 - 1.5 * IQR
    upper_limit = Q3 + 1.5 * IQR

    # 3. Filter the dataframe to show only these rows
    outliers = df[(df[col_x] < lower_limit) | (df[col_x] > upper_limit)]

    # Show the results
    print(f"Found {len(outliers)} outliers. Upper limit is: {upper_limit}")
    display(outliers.sort_values(by=col_x, ascending=False))

n_outliers(df, 'house_size')

Found 6294 outliers. Upper limit is: 4120.0


,status,bed,bath,acre_lot,city,state,zip_code,house_size,price,has_prev_sale,years_since_prev_sale
132744,sold,20,20,0.42,Birmingham,Alabama,35206,50000.000000,399900.0,1,3.884932
97904,for_sale,3,4,21.00,Lahaina,Hawaii,96761,50000.000000,295000.0,0,0.000000
130454,for_sale,20,20,23.00,Marion,Texas,78124,50000.000000,200000.0,0,0.000000
2281,for_sale,2,2,6.67,Lahaina,Hawaii,96761,50000.000000,18900.0,1,6.649315
26897,for_sale,3,2,2.00,Montague,California,96064,50000.000000,285000.0,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
79146,ready_to_build,4,3,0.21,Middletown,Delaware,19709,4121.000000,778900.0,0,0.000000
95252,sold,5,4,0.31,Encinitas,California,92024,4121.000000,2850000.0,1,3.600000
88729,sold,4,4,0.27,Sugar Land,Texas,77479,4121.000000,700000.0,1,3.827397
104109,sold,4,5,10.00,East Amwell Township,New Jersey,8551,4120.478644,675000.0,1,3.668493


In [219]:
df['house_size'].describe(percentiles=[0.90, 0.95, 0.99, 0.999])


count    140951.000000
mean       2039.080874
std        1348.297662
min         100.000000
50%        1757.000000
90%        3280.000000
95%        4000.000000
99%        6421.700847
99.9%     14381.000000
max       50000.000000
Name: house_size, dtype: float64

In [220]:
outliers.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price,has_prev_sale,years_since_prev_sale
4,sold,4,3,0.42,Wilmington,Delaware,19807,4327.0,600000.0,1,3.904110
11,for_sale,4,3,0.27,Round Hill,Virginia,20141,4864.0,850000.0,1,8.430137
13,sold,5,5,0.21,Coto de Caza,California,92679,4223.0,2188000.0,1,3.610959
107,sold,4,4,0.21,Fredericksburg,Virginia,22406,4982.0,525000.0,1,4.019178
147,for_sale,5,6,2.61,The Woodlands,Texas,77380,6887.0,1750000.0,1,24.610959


In [221]:
df = df[(df['house_size'] >= lower_limit) & (df['house_size'] <= upper_limit)]


In [222]:
df['house_size'].describe(percentiles=[0.90, 0.95, 0.99, 0.999])


count    134657.000000
mean       1853.078715
std         758.840630
min         100.000000
50%        1713.000000
90%        2957.000000
95%        3347.000000
99%        3909.000000
99.9%      4100.000000
max        4120.000000
Name: house_size, dtype: float64

In [223]:
df['house_size_log'] = np.log1p(df['house_size'])


In [224]:
df['acre_lot'].describe(percentiles=[0.5, 0.75, 0.95, 0.99, 0.999])

count    134657.000000
mean         10.485616
std         714.127477
min           0.000000
50%           0.210000
75%           0.360000
95%           3.000000
99%          21.500000
99.9%       246.688000
max      100000.000000
Name: acre_lot, dtype: float64

In [225]:
n_outliers(df, 'acre_lot')

Found 19685 outliers. Upper limit is: 0.675


,status,bed,bath,acre_lot,city,state,zip_code,house_size,price,has_prev_sale,years_since_prev_sale,house_size_log
71018,sold,2,2,100000.00,Alhambra,California,91801,1110.000000,598000.0,1,3.594521,7.013016
121429,for_sale,3,3,100000.00,Islamorada,Florida,33036,2026.000000,1995000.0,0,0.000000,7.614312
107254,sold,3,2,100000.00,Simpsonville,South Carolina,29680,1991.589489,159800.0,1,3.756164,7.597190
72737,for_sale,3,1,100000.00,Sanford,North Carolina,27330,1250.000000,364000.0,1,25.038356,7.131699
51670,sold,1,1,85442.00,San Diego,California,92108,682.000000,355000.0,1,3.978082,6.526495
...,...,...,...,...,...,...,...,...,...,...,...,...
81364,sold,3,2,0.68,Odessa,Florida,33556,1490.000000,599000.0,1,3.668493,7.307202
49693,for_sale,3,1,0.68,Collierville,Tennessee,38017,883.518946,279000.0,1,11.495890,6.785044
93370,for_sale,4,3,0.68,Walden,New York,12586,2318.000000,469900.0,1,9.010959,7.748891
93394,sold,9,7,0.68,Chadbourn,North Carolina,28431,3650.000000,169000.0,1,3.695890,8.202756


In [226]:
noutliers = len(df[df['acre_lot']>3])
print(noutliers)
df = df[df['acre_lot'] < 3]

6725


In [227]:
df['acre_lot_log'] = np.log1p(df['acre_lot'])

df['price_log'] = np.log1p(df['price'])


## Encoding.

In [228]:
from category_encoders import TargetEncoder

encoder = TargetEncoder(cols=['zip_code', 'city', 'state', 'status'])

encoded = encoder.fit_transform(
    df[['zip_code', 'city', 'state', 'status']], 
    df['price']
)

df['zip_code_encoded'] = encoded['zip_code']
df['city_encoded'] = encoded['city']
df['state_encoded'] = encoded['state']
df['status_encoded'] = encoded['status']

# frequency of the features 
df['zip_freq'] = df['zip_code'].map(df['zip_code'].value_counts())
df['city_freq'] = df['city'].map(df['city'].value_counts())

In [229]:
# # drop the coloumns after encofing
# df = df.drop(columns=['city', 'zip_code', 'state', 'status'])

In [230]:
df['bed_bath_interactions'] = df['bed']*df['bath']
df["size_per_bed"] = df["house_size"] / df["bed"].replace(0, 1)
df["bath_per_bed"] = df["bath"] / df["bed"].replace(0, 1)
df["size_bath_interaction"] = df["house_size"] * df["bath"]

In [231]:
# corr_mat = df.corr()
# corr_mat['price'].sort_values(ascending=True)

In [232]:
df.to_csv(r'c:\Users\Администратор\Documents\study\CSME 830\house_price_project\models\re_cleaned.csv')


In [244]:
import sys
import os

# go up to the project root (the folder that contains /src)
project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.append(project_root)

print(sys.path)


['c:\\Users\\Администратор\\Documents\\study\\CSME 830\\house_price_project\\notebooks', 'c:\\anaconda\\python312.zip', 'c:\\anaconda\\DLLs', 'c:\\anaconda\\Lib', 'c:\\anaconda', '', 'c:\\anaconda\\Lib\\site-packages', 'c:\\anaconda\\Lib\\site-packages\\win32', 'c:\\anaconda\\Lib\\site-packages\\win32\\lib', 'c:\\anaconda\\Lib\\site-packages\\Pythonwin', 'c:\\anaconda\\Lib\\site-packages\\setuptools\\_vendor', 'c:\\Users\\Администратор\\Documents\\study\\CSME 830\\house_price_project']


In [233]:
from sklearn.base import BaseEstimator, TransformerMixin

class FrequencyEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        self.freq_maps = {}

    def fit(self, X, y=None):
        for col in self.columns:
            self.freq_maps[col] = X[col].value_counts().to_dict()
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.columns:
            X[col + "_freq"] = X[col].map(self.freq_maps[col]).fillna(0)
        return X


In [245]:
# class InteractionFeatures(BaseEstimator, TransformerMixin):
#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         X = X.copy()

#         # Interaction features using your real columns
#         X["bed_bath_interactions"] = X["bed"] * X["bath"]
#         X["size_per_bed"] = X["house_size"] / (X["bed"] + 1)
#         X["bath_per_bed"] = X["bath"] / (X["bed"] + 1)
#         X["size_bath_interaction"] = X["house_size"] * X["bath"]
#         X["acre_lot_log"] = np.log1p(X["acre_lot"])

#         return X
from src.features.custom_features import InteractionFeatures

In [246]:
from sklearn.pipeline import Pipeline
from category_encoders import TargetEncoder
from catboost import CatBoostRegressor

categorical_cols = ['zip_code', 'city', 'state', 'status']
freq_cols = ['zip_code', 'city']

pipeline = Pipeline([
    ("interactions", InteractionFeatures()),   # <-- NEW
    ("freq_enc", FrequencyEncoder(columns=freq_cols)),
    ("target_enc", TargetEncoder(cols=categorical_cols)),
    ("model", CatBoostRegressor(
        depth=10,
        learning_rate=0.05,
        n_estimators=1000,
        loss_function='RMSE',
        verbose=False
    ))
])

In [247]:
X = df[['status', 'bed', 'bath', 'acre_lot', 'city', 'state', 'zip_code', 'house_size']]
y = df['price']

pipeline.fit(X, y)


,steps,"[('interactions', ...), ('freq_enc', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,columns,"['zip_code', 'city']"
,verbose,0
,cols,"['zip_code', 'city', ...]"
,drop_invariant,False
,return_df,True
,handle_missing,'value'
,handle_unknown,'value'


In [248]:
import joblib
joblib.dump(pipeline, "catboost_full_pipeline.pkl")


['catboost_full_pipeline.pkl']

In [252]:
pipeline = joblib.load("catboost_full_pipeline.pkl")

new_data = pd.DataFrame({
    "status": ["for_sale"],
    "bed": [3],
    "bath": [2],
    "acre_lot": [0.20],
    "city": ["Seattle"],
    "state": ["Washington"],
    "zip_code": ["98109"],
    "house_size": [2000]
})

prediction = pipeline.predict(new_data)
print(prediction)


[635725.78142564]


In [237]:
features = [
    "price",
    # "bed",
    # "bath",
    "bed_bath_interactions",
    "house_size",
    # "house_size_log",
    "acre_lot_log",
    "zip_code_encoded",
    "city_encoded",
    "state_encoded",
    "zip_freq",
    "city_freq",
    "size_per_bed",
    "bath_per_bed",
    "size_bath_interaction"
]

df_model = df[features]

In [238]:
# from catboost import CatBoostRegressor
# from sklearn.model_selection import train_test_split

# # Target
# X = df_model.drop(columns=['price']) 
# y = np.log1p(df_model["price"])

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.2, random_state=42
# )
# print("Split is complete.")

In [ ]:
# # Model
# model = CatBoostRegressor(
#     iterations=3000,
#     depth=10,
#     learning_rate=0.03,
#     l2_leaf_reg=5,
#     loss_function="RMSE",
#     bagging_temperature=0.5,
#     random_seed=42,
#     verbose=200
# )


# # Train
# model.fit(X_train, y_train)
# print('model training is complete.')
# # Score
# print("Train R2:", model.score(X_train, y_train))
# print("Test R2:", model.score(X_test, y_test))
